In [1]:
import pytket as tk
from pytket import passes as tkp
from pytket import Circuit
from pytket.circuit.display import render_circuit_jupyter as print_circ
import numpy as np
from pytket import OpType

In [5]:
def h_gadgetisation2(circ: Circuit):
    n_qubits = circ.n_qubits #original # of qubits
    h = circ.n_gates_of_type(OpType.H) # of H gates

    new_circ = Circuit(n_qubits+h) #Define a replacement circuit

    for cmd in circ.get_commands():
        qa = n_qubits # ancilla qubit
        qubit_list = cmd.qubits # accesing the qubit of cmd
        if cmd.op.type == OpType.H:
            q1 = qubit_list[0].index[0] # we want the number
            # If cmd is a H gate, do H-gadgetisation
            new_circ.S(q1).S(qa).CX(q1, qa).Sdg(qa).CX(qa,q1).CX(q1,qa)
            qa += 1 
        else:
            # Otherwise, apply the gate as usual.
            new_circ.add_gate(cmd.op.type, cmd.op.params, qubit_list)

    return new_circ

In [7]:
DecompseZPass = tkp.CustomPass(h_gadgetisation2) # Define our pass

test_circ = tk.Circuit(5).H(1).CX(2,3).CX(2,1).CY(1,2).H(2).Z(3)
print_circ(test_circ)

DecompseZPass.apply(test_circ) # Apply our pass to the test Circuit

test_circ.get_commands()
print_circ(test_circ)